# Dataset Generation

Here we have to do the following:
- Load all the wav files from the corpus
  - Separate out the individual instruments with the audio itself
    - Separate out the individual instruments and convert to MIDI
    - Verify the MIDI data for each
- Creating of sections based on the timestamps
  - From the MIDI data, loop through and make sure that each note is in the correct section with the correct label
- Loading of the metadata from the corpus as well (we have differing meta-data based on the captioning types of things here)
- Any other data or notes will be here with everything else

In [ ]:
!pip install torch
!pip install basic-pitch
!pip install music21
!pip install tensorflow
!pip install librosa
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

## Separating out the individual stems

In [15]:
model = "htdemucs_ft"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.

in_path = '../audio/'
out_path = '../audio/stems/'

In [16]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

# from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')
    
    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()
    
    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()
    
    separate(in_path, out_path)


In [ ]:
separate()

In [ ]:
import glob
import librosa
from basic_pitch.inference import predict_and_save, Model
from basic_pitch import ICASSP_2022_MODEL_PATH

basic_pitch_model = Model(ICASSP_2022_MODEL_PATH)

audio_directory = '../audio/test/'
audio_files = glob.glob(audio_directory + '*.mp3')

for audio_file in audio_files:
    # Estimate BPM using librosa
    print(audio_file)
    y, sr = librosa.load(audio_file)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    
    tempo = int(round(tempo[0]))
    print(f"Tempo: {tempo}")

    predict_and_save(
        audio_path_list=[audio_file],
        output_directory='../midi/',
        save_model_outputs=True,
        save_midi=True,
        sonify_midi=True,
        save_notes=True,
        minimum_note_length=127.70,
        model_or_model_path=basic_pitch_model,
        onset_threshold=0.5,
        frame_threshold=0.3,
        sonification_samplerate=44100,
        midi_tempo=tempo  # Use estimated tempo here
    )

In [ ]:
import music21
import matplotlib.pyplot as plt
%matplotlib inline

midi_file = music21.converter.parse('../midi/01 - Ravi Krishnaswami - 0598_copilot_basic_pitch.mid')

In [ ]:
midi_file.show('midi')

In [ ]:
midi_file.plot()